In [ ]:
import nltk
import pandas as pd
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df = pd.read_json("dataframe.json")